# Preparing Experiments

In this notebook, we will prepare several parts of our experiments including
deploying and configuring our workload, loading traces, and creating functions
to be called to generate the workloads.

In [1]:
from tqdm.auto import tqdm
import itertools

import os

# my imports
from helpers import kube
from helpers import workload
from helpers import util
from helpers import request_funcs

fetching imagenet v2
resizing images


  0%|          | 0/100 [00:00<?, ?it/s]

converting to bentoml files


  0%|          | 0/100 [00:00<?, ?it/s]

extracting base64 files


  0%|          | 0/100 [00:00<?, ?it/s]

preprocessing for mobilenet


  0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
# making sure kn is set up correctly
!kn service ls

NAME                    URL                                                    LATEST                        AGE     CONDITIONS   READY   REASON
autoscale-go            http://autoscale-go.default.kn.nima-dev.com            autoscale-go-00035            91d     3 OK / 3     True    
bench1                  http://bench1.default.kn.nima-dev.com                  bench1-00054                  100d    3 OK / 3     True    
bentoml-iris            http://bentoml-iris.default.kn.nima-dev.com            bentoml-iris-00003            2d21h   3 OK / 3     True    
bentoml-onnx-resnet50   http://bentoml-onnx-resnet50.default.kn.nima-dev.com   bentoml-onnx-resnet50-00002   2d21h   3 OK / 3     True    
helloworld-go           http://helloworld-go.default.kn.nima-dev.com           helloworld-go-00001           101d    3 OK / 3     True    
tfserving-mobilenetv1   http://tfserving-mobilenetv1.default.kn.nima-dev.com   tfserving-mobilenetv1-00002   2d20h   3 OK / 3     True    
tfserving-resnetv2   

In [3]:
EXP_CONFIG_NAME_DEFAULT = 'bentoml-iris-250m-512mb'
# EXP_CONFIG_NAME_DEFAULT = 'bentoml-onnx-resnet50-250m-512mb'
# EXP_CONFIG_NAME_DEFAULT = 'tfserving-resnetv2-250m-512mb'
# EXP_CONFIG_NAME_DEFAULT = 'tfserving-mobilenetv1-250m-512mb'
exp_config_name = os.getenv("DEPLOYMENT_CONFIG_NAME", EXP_CONFIG_NAME_DEFAULT)
exp_file = f"deployments/{exp_config_name}.json"

workload_spec = util.load_json_file(exp_file)

kn_command = kube.get_kn_command(**workload_spec)
print(kn_command)
# to run the command, we can simply use:
# !{kn_command}
# or from python:
# get_ipython().system('{kn_command}')

kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=250m,memory=512Mi' \
  --request 'cpu=250m,memory=512Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency


In [4]:
# deploy the workload
!{kn_command}

No changes to apply to service 'bentoml-iris'.
Service 'bentoml-iris' with latest revision 'bentoml-iris-00003' (unchanged) is available at URL:
http://bentoml-iris.default.kn.nima-dev.com


In [5]:
# testing the workload functions
!python helpers/request_funcs.py

2021-05-03 15:05:30.551174: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-05-03 15:05:30.551216: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-05-03 15:05:33.793577: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-03 15:05:33.793851: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-05-03 15:05:33.793886: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-05-03 15:05:33.793924: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 